In [3]:
import tensorflow as tf
import tensorflow_probability as tfp

The differentiable simulation techniques require z-axis differentiation, which usually would be converted to an index into a tensor along the drift distance.  That's not differentiable.  Here, this shows how to use gaussian responses to provide a nearly equivalent response that is differentiable along z, at the cost of some performance.

In [83]:
MAX_TICKS=40
N_SENSORS=12
z = tf.convert_to_tensor([10.1,15.5, 25.4])
response = tf.random.uniform(shape=(len(z), 12))

total_result = tf.zeros((N_SENSORS, MAX_TICKS))

print(response.shape)
print(total_result.shape)

(3, 12)
(12, 40)


In [84]:
test_range = tf.range(40)

In [85]:
empty_ticks = tf.stack([ tf.range(40, dtype=tf.float32) for _z in z])
print(empty_ticks)
print(empty_ticks.shape)

tf.Tensor(
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
  18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
  36. 37. 38. 39.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
  18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
  36. 37. 38. 39.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
  18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
  36. 37. 38. 39.]], shape=(3, 40), dtype=float32)
(3, 40)


In [86]:
z = tf.reshape(z, z.shape + (1,))
print(z)

tf.Tensor(
[[10.1]
 [15.5]
 [25.4]], shape=(3, 1), dtype=float32)


In [96]:
z_values = tf.exp( -(empty_ticks - z)**2 / 0.1)
print(z_values)

tf.Tensor(
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 7.0405362e-20 5.5594637e-06
  9.0483677e-01 3.0354134e-04 2.0988263e-16 2.9911947e-37 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 7.1877814e-28 1.6918979e-10
  8.2084998e-02 8.2084998e-02 1.6918979e-10 7.1877814e-28 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.000000

In [108]:
print(tf.math.argmax(z_values, axis=-1))

tf.Tensor([10 15 25], shape=(3,), dtype=int64)


The values come out in the right spots!  Add the sensor responses:

In [109]:
print(z_values.shape)
print(response.shape)

(3, 40)
(3, 12)


In [110]:
test_result = tf.transpose(tf.linalg.matmul(tf.transpose(z_values), response))

In [107]:
print(test_result.shape)
print(tf.math.argmax(test_result, axis=1))

(12, 40)
tf.Tensor([10 10 10 15 10 10 25 10 10 10 10 25], shape=(12,), dtype=int64)


In [75]:
response = tf.random.uniform(shape=(len(z), 40))

In [76]:
print(response.shape)

(1, 40)


Can we do it with sparse tensors?

In [117]:
sparse_z_values_t = tf.sparse.from_dense(tf.transpose(z_values))

In [118]:
print(sparse_z_values)

SparseTensor(indices=tf.Tensor(
[[ 0  8]
 [ 0  9]
 [ 0 10]
 [ 0 11]
 [ 0 12]
 [ 0 13]
 [ 1 13]
 [ 1 14]
 [ 1 15]
 [ 1 16]
 [ 1 17]
 [ 1 18]
 [ 2 23]
 [ 2 24]
 [ 2 25]
 [ 2 26]
 [ 2 27]
 [ 2 28]], shape=(18, 2), dtype=int64), values=tf.Tensor(
[7.0405362e-20 5.5594637e-06 9.0483677e-01 3.0354134e-04 2.0988263e-16
 2.9911947e-37 7.1877814e-28 1.6918979e-10 8.2084998e-02 8.2084998e-02
 1.6918979e-10 7.1877814e-28 9.6526179e-26 3.0749139e-09 2.0189714e-01
 2.7323594e-02 7.6217747e-12 4.3821147e-30], shape=(18,), dtype=float32), dense_shape=tf.Tensor([ 3 40], shape=(2,), dtype=int64))


In [122]:
test_result = tf.transpose(tf.sparse.sparse_dense_matmul(sparse_z_values_t, response))

In [123]:
print(test_result.shape)

(12, 40)
